In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. 환경 설정(Environment Setting)

## 1.1 Library Import

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# 데이터 읽기를 위한 라이브러리
from pathlib import Path
import numpy as np
np.random.seed(0)
import pandas as pd
import math
import gc, os, time
import scipy as sp
import statsmodels.api as sm
from pandas import DataFrame, Series
from datetime import datetime, date, timedelta
from sklearn.preprocessing import StandardScaler, LabelEncoder

# 탐색적 데이터 분석을 위한 라이브러리
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import skew, norm, probplot, boxcox
from scipy.special import boxcox1p

# 모델링을 위한 라이브러리
from sklearn.linear_model import Ridge, LinearRegression, ElasticNet, Lasso, LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_selection import SelectKBest, mutual_info_regression, VarianceThreshold, SelectFromModel, RFE, RFECV
from sklearn.svm import SVR
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from mlxtend.regressor import StackingCVRegressor


## 1.2 모델 성능을 확인하기 위한 함수

In [ ]:
# 모듈 설치
!pip install teddynote -q
from teddynote import utils

# 그래프 사이즈 설정
utils.set_plot_options(figsize=(10, 4))

# MSE 에러 설정
utils.set_plot_error('rmse')

df_results = pd.DataFrame()

## 1.3 DataFrame Option 설정

In [ ]:
pd.set_option("display.max_colwidth", 999)  # 컬럼 정보 보여주기
pd.set_option("display.max_rows", 150)  # row 정보 보여주기
# pd.reset_option("display.max_colwidth")
# 참고 : https://pandas.pydata.org/docs/user_guide/options.html

# 2. 데이타 읽기(Data Loading)

## 2.1 데이타 읽기

In [ ]:
data_dir = Path("../input/house-prices-advanced-regression-techniques/")
train = pd.read_csv(data_dir / "train.csv")  # pd.read_csv(data_dir / "train.csv", index_col="Id")
test = pd.read_csv(data_dir / "test.csv") # pd.read_csv(data_dir / "test.csv", index_col="Id")

In [ ]:
train.shape, test.shape

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
list(test['Street'].value_counts().index)

## 2.2 데이타 설명

In [ ]:
# Dataframe의 데이타 구성 특성을 조회한다.
def rstr(df, pred=None): 
    
    obs = df.shape[0]
    types = df.dtypes # 자료형 
    counts = df.apply(lambda x: x.count()) # 값의 갯수 
    uniques = df.apply(lambda x: x.unique()) # 고유값 
    distincts = df.apply(lambda x: x.unique().shape[0]) # 고유값의 갯수 
    nulls = df.apply(lambda x: x.isnull().sum()) # 결측치의 갯수  
    nulls_ratio = (df.isnull().sum()/ obs) * 100 # 결측 비율 
    skewness = df.skew() # 왜도  
    kurtosis = df.kurt() # 첨도  
    
    print('Data shape:', df.shape)
    
    if pred is None:
        cols = ['types', 'counts', 'distincts', 'nulls', 'nulls_ratio', 'uniques', 'skewness', 'kurtosis']
        str = pd.concat([types, counts, distincts, nulls, nulls_ratio, uniques, skewness, kurtosis], axis = 1)

    else:
        corr = df.corr()[pred]
        str = pd.concat([types, counts, distincts, nulls, nulls_ratio, uniques, skewness, kurtosis, corr], axis = 1, sort=False)
        corr_col = 'corr '  + pred
        cols = ['types', 'counts', 'distincts', 'nulls', 'nulls_ratio', 'uniques', 'skewness', 'kurtosis', corr_col ]

    str.columns = cols
    dtypes = str.types.value_counts()
    print('___________________________\nData types:\n',dtypes)
    print('___________________________')
    
    return str

In [ ]:
details = rstr(train, 'SalePrice')
display(details.sort_values(by='corr SalePrice', ascending=False))

In [ ]:
# Train, Test Feature 특성 비교하기
def compare_cols(df_train, df_test, target='y'):
    ''' train vs test information compare '''
    
    # train의 target column을 제외하여 Test의 column과 동일하게 처리
    df_train_drop = df_train.drop(target, axis=1)
    
    train_obs = df_train_drop.shape[0]
    train_types = df_train_drop.dtypes # 자료형 
    train_counts = df_train_drop.apply(lambda x: x.count()) # 값의 갯수 
    train_uniques = df_train_drop.apply(lambda x: x.unique()) # 고유값 
    train_distincts = df_train_drop.apply(lambda x: x.unique().shape[0]) # 고유값의 갯수 
    train_nulls = df_train_drop.apply(lambda x: x.isnull().sum()) # 결측치의 갯수  
    train_nulls_ratio = (df_train_drop.isnull().sum()/ train_obs) * 100 # 결측 비율 
    train_skewness = df_train_drop.skew() # 왜도  
    train_kurtosis = df_train_drop.kurt() # 첨도  
    
    test_obs = df_test.shape[0]
    test_types = df_test.dtypes # 자료형 
    test_counts = df_test.apply(lambda x: x.count()) # 값의 갯수 
    test_uniques = df_test.apply(lambda x: x.unique()) # 고유값 
    test_distincts = df_test.apply(lambda x: x.unique().shape[0]) # 고유값의 갯수 
    test_nulls = df_test.apply(lambda x: x.isnull().sum()) # 결측치의 갯수  
    test_nulls_ratio = (df_test.isnull().sum()/ test_obs) * 100 # 결측 비율 
    test_skewness = df_test.skew() # 왜도  
    test_kurtosis = df_test.kurt() # 첨도  
    
    cols = ['types', 't_types', 'same_type', 'counts', 't_counts', 'nulls', 't_nulls', 'nulls_ratio', 't_null_ratio', 
            'distincts', 't_distincts', 'uniques', 't_uniques', 'skew', 't_skew', 'kurtosis', 't_kurtosis']
    str = pd.concat([train_types, test_types, train_types == test_types, train_counts, test_counts, train_nulls, test_nulls, train_nulls_ratio, test_nulls_ratio, 
                     train_distincts, test_distincts, train_uniques, test_uniques, train_skewness, test_skewness, train_kurtosis, test_kurtosis], axis = 1)

    str.columns = cols

    return str

In [ ]:
df_comp = compare_cols(train, test, 'SalePrice')
display(df_comp.sort_values(['same_type','nulls_ratio','t_null_ratio'], ascending=[True, False,False]))

In [ ]:
# skew train data > 0.5
condt = abs(df_comp['skew']) > 0.35   # 0.4 vs 0.35 same
col_skewed_0point5 = set(df_comp.loc[condt].index)
print(col_skewed_0point5)

In [ ]:
# skew predict data > 0.5
condp = abs(df_comp['t_skew']) > 0.35  # 0.4 vs 0.35 same
tcol_skewed_0point5 = set(df_comp.loc[condp].index)
print(tcol_skewed_0point5)

In [ ]:
# skew train union predict data > 0.5
col_union_skewed_0point5 = col_skewed_0point5 | tcol_skewed_0point5
print(col_union_skewed_0point5)

In [ ]:
# 조회 기본 조건
cond1 = df_comp['same_type'] == False # type이 다르다
cond2 = df_comp['types']     == object # type이 'object' 이다.

In [ ]:
cond = cond1
df_comp.loc[cond]

In [ ]:
cond = ~(cond1) & (cond2)
df_comp.loc[cond]

In [ ]:
train.info(), test.info()

# 3. 탐색적 데이타분석 (EDA)

## 3.1 탐색적 데이타분석 준비

### 1) Train Data 기초 분석

In [ ]:
%%time

# EDA를 위한 학습 데이터를 복사합니다.
df_eda = train.copy()

In [ ]:
# 숫자형 데이타 특징 조회하기
df_eda.describe()

In [ ]:
# 무 결측치 숫자형 컬럼 정보
col_nona_numerical = list(df_eda.describe().T[df_eda.describe().T['count'] == df_eda.shape[0]].T.columns)
print(col_nona_numerical)
# 무 결측치 데이타 분포 점검하기
df_eda.describe().T[df_eda.describe().T['count'] == df_eda.shape[0]].T

In [ ]:
# 유 결측치 숫자형 컬럼 정보
col_hasna_numerical = list(df_eda.describe().T[df_eda.describe().T['count'] < df_eda.shape[0]].T.columns)
print(col_hasna_numerical)
# 유 결측치 데이타 분포 점검하기
df_eda.describe().T[df_eda.describe().T['count'] < df_eda.shape[0]].T

In [ ]:
df_eda.describe().T[df_eda.describe().T['count'] < df_eda.shape[0]][['mean','max','75%']].T
# df_eda.describe().T[df_eda.describe().T['count'] < df_eda.shape[0]]['75%'].T['LotFrontage']

In [ ]:
# Object 데이타 특징 조회하기
df_eda.describe(include='object')

In [ ]:
# 무 결측치 object형 컬럼 정보
col_nona_object = list(df_eda.describe(include='object').T[df_eda.describe(include='object').T['count'] == df_eda.shape[0]].T.columns)
print(col_nona_object)
# 무 결측치 Object 데이타 분포 점검하기
df_eda.describe(include='object').T[df_eda.describe(include='object').T['count'] == df_eda.shape[0]].T

In [ ]:
# 유 결측치 object형 컬럼 정보
col_hasna_object = list(df_eda.describe(include='object').T[df_eda.describe(include='object').T['count'] < df_eda.shape[0]].T.columns)
print(col_hasna_object)
# 유 결측치 Object 데이타 분포 점검하기
df_eda.describe(include='object').T[df_eda.describe(include='object').T['count'] < df_eda.shape[0]].T

In [ ]:
df_eda.describe(include='object').T[df_eda.describe(include='object').T['count'] < df_eda.shape[0]]['top'].T
# df_eda.describe(include='object').T[df_eda.describe(include='object').T['count'] < df_eda.shape[0]]['top'].T['Alley']

### 2) Predict Data 기초 분석

In [ ]:
# EDA를 위한 평가 데이터를 복사합니다.
df_eda_predict = test.copy()

In [ ]:
# 무 결측치 숫자형 컬럼 정보
col_nona_numerical_predict = list(df_eda_predict.describe().T[df_eda_predict.describe().T['count'] == df_eda_predict.shape[0]].T.columns)
print(col_nona_numerical_predict)
# 무 결측치 데이타 분포 점검하기
df_eda_predict.describe().T[df_eda_predict.describe().T['count'] == df_eda_predict.shape[0]].T

In [ ]:
# 유 결측치 숫자형 컬럼 정보
col_hasna_numerical_predict = list(df_eda_predict.describe().T[df_eda_predict.describe().T['count'] < df_eda_predict.shape[0]].T.columns)
print(col_hasna_numerical_predict)
# 유 결측치 데이타 분포 점검하기
df_eda_predict.describe().T[df_eda_predict.describe().T['count'] < df_eda_predict.shape[0]].T

In [ ]:
# 무 결측치 object형 컬럼 정보
col_nona_object_predict = list(df_eda_predict.describe(include='object').T[df_eda_predict.describe(include='object').T['count'] == df_eda_predict.shape[0]].T.columns)
print(col_nona_object_predict)
# 무 결측치 Object 데이타 분포 점검하기
df_eda_predict.describe(include='object').T[df_eda_predict.describe(include='object').T['count'] == df_eda_predict.shape[0]].T

In [ ]:
# 유 결측치 object형 컬럼 정보
col_hasna_object_predict = list(df_eda_predict.describe(include='object').T[df_eda_predict.describe(include='object').T['count'] < df_eda_predict.shape[0]].T.columns)
print(col_hasna_object_predict)
# 유 결측치 Object 데이타 분포 점검하기
df_eda_predict.describe(include='object').T[df_eda_predict.describe(include='object').T['count'] < df_eda_predict.shape[0]].T

In [ ]:
# 시간과 관련한 분석을 진행하기 위하여 날짜형으로 변환합니다. 
# df_eda[col_time] = df_eda[col_time].apply(pd.to_datetime)

### 3.1.1 결측치 보정 전 신규 Feature 생성

In [ ]:
# flag [True, False] categry field 추가
flag_fields = ['Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Electrical', 'FireplaceQu', 'GarageType', 'PoolQC', 'Fence', 'MiscFeature']

In [ ]:
def add_flagfields(df):
    for ffield in flag_fields:
        col_name = 'gen_existflag_' + ffield
        if not col_name in df.columns:
            #df[col_name] = df[ffield].notna()     # Boolean type
            df[col_name] = df[ffield].apply(lambda x : 1 if type(x) == type('str') else 0)  # 1, 0 numeric type, chart display 
            print(col_name)
        else:
            print(f'exits column : {col_name}')

In [ ]:
add_flagfields(df_eda)
# 추가 flag field 변수
col_gen_flagfields = list(df_eda.filter(regex='gen_existflag_').columns)
df_eda[col_gen_flagfields].describe()

### 3.1.2 결측치 보정

In [ ]:
#col_nona_numerical
#col_hasna_numerical
#col_nona_object
#col_hasna_object
#col_nona_numerical_predict
#col_hasna_numerical_predict
#col_nona_object_predict
#col_hasna_object_predict
#col_gen_flagfields

In [ ]:
# Numerical의 결측치 보정하기
def df_numerical_fillna(df, data='Train'):
    if data == 'Train' : # Train data
        for col_num in col_hasna_numerical:  # 숫자형은 상위 75% 값으로
            if df[col_num].isna().sum() > 0 :
                df.loc[:,col_num] = df.loc[:,col_num].fillna(df.describe().T[df.describe().T['count'] < df.shape[0]]['75%'].T[col_num]) #'mean','max','75%'
    else : # Predict data
        for col_num in col_hasna_numerical_predict:  # 숫자형은 상위 75% 값으로
            if df[col_num].isna().sum() > 0 :
                df.loc[:,col_num] = df.loc[:,col_num].fillna(df.describe().T[df.describe().T['count'] < df.shape[0]]['75%'].T[col_num]) #'mean','max','75%'        
    return df

In [ ]:
# 참고자료 : Object를 top 빈도의 값으로 치환하는 방법 예시
#     for col_obj in col_hasna_object: # Object 형은 top 구분 값으로
#         if df[col_obj].isna().sum() > 0 :
#             df.loc[:,col_obj] = df.loc[:,col_obj].fillna(df.describe(include='object').T[df.describe(include='object').T['count'] < df.shape[0]]['top'].T[col_obj])
def conv_obj2num(items, x):
    if type(x) != type('str') :
        return 0
    elif x in items :
        return items.index(x) + 1
    else :
        return 0
# Object의 결측치 보정 및 Numeric 값으로 치환 , Object의 unique는 Test 값의 unique도 포함하도록 하여 학습 결과 예측 시 Numeric 치환 값 동일하게 적용 목적 
def df_object_fillna(df, df2, data='Train'):
    if data == 'Train' : # Train data
        for col_obj in col_hasna_object:
            items = np.sort(list(set(list(df[col_obj].unique())) | set(list(df2[col_obj].unique()))))
            df[col_obj] = df[col_obj].apply(lambda x : conv_obj2num(list(items), x))
        for col_obj in col_nona_object:
            items = np.sort(list(set(list(df[col_obj].unique())) | set(list(df2[col_obj].unique()))))
            df[col_obj] = df[col_obj].apply(lambda x : conv_obj2num(list(items), x))
    else: # Predict data
        for col_obj in col_hasna_object_predict:
            items = np.sort(list(set(list(df[col_obj].unique())) | set(list(df2[col_obj].unique()))))
            df[col_obj] = df[col_obj].apply(lambda x : conv_obj2num(list(items), x))
        for col_obj in col_nona_object_predict:
            items = np.sort(list(set(list(df[col_obj].unique())) | set(list(df2[col_obj].unique()))))
            df[col_obj] = df[col_obj].apply(lambda x : conv_obj2num(list(items), x))       
    return df

In [ ]:
# numerical feature 결측치 보정하기
df_numerical_fillna(df_eda, 'Train')

In [ ]:
# object feature 결측치 보정 및 숫자형 변환
df_object_fillna(df_eda, test, 'Train')

### 3.1.3 결측치 보정 후 신규 Feature 추가

#### 1) 날짜 Field를 그룹핑하여 추가 컬럼 만들기

In [ ]:
# fields 중 Datetime 유형만 추출
datefields = [{'Name':'YearBuilt'   , 'DType':'int64'  ,'Type' : 'year' , 'Scale' : 5},
             {'Name':'YearRemodAdd', 'DType':'int64'  ,'Type' : 'year' , 'Scale' : 5},
             {'Name':'GarageYrBlt' , 'DType':'float64','Type' : 'year' , 'Scale' : 5},
             {'Name':'YrSold'      , 'DType':'int64'  ,'Type' : 'year' , 'Scale' : 5},
             {'Name':'MoSold'      , 'DType':'int64'  ,'Type' : 'month', 'Scale' : 3},
            ]
def data_datefields(df):
    cols = []
    for dfield in datefields:
        cols.append(dfield['Name'])
    print(cols)
    return df[cols]
def groupbyscale(x, scale):
    if scale <= 1 :
        return x
    else :
        result_mod = x // scale
        return result_mod * scale
def add_datafields(df):
    for dfield in datefields:
        col_name = 'gen_grpdate_' + dfield['Name']
        if not col_name in df.columns:
            df[col_name] = df[dfield['Name']].apply(lambda x : groupbyscale(x, dfield['Scale']))
            print(col_name)
        else:
            print(f'exits column : {col_name}')
def get_alldatecols(df):
    results = list(df.filter(regex='gen_grpdate_').columns)
    for dfield in datefields:
        results.append(dfield['Name'])
    return results

In [ ]:
data_datefields(df_eda)

In [ ]:
# 추가 date field 생성
add_datafields(df_eda) 
# 추가 date field 변수
col_gen_datefields = list(df_eda.filter(regex='gen_grpdate_').columns)
print(col_gen_datefields)

In [ ]:
df_eda.filter(regex='gen_').columns

In [ ]:
df_eda[get_alldatecols(df_eda)]

In [ ]:
print(f'col_nona_numerical : {col_nona_numerical}')
print(f'col_nona_numerical_predict : {col_nona_numerical_predict}')
print(f'col_hasna_numerical : {col_hasna_numerical}')
print(f'col_hasna_numerical_predict : {col_hasna_numerical_predict}')
print(f'col_nona_object : {col_nona_object}')
print(f'col_nona_object_predict : {col_nona_object_predict}')
print(f'col_hasna_object : {col_hasna_object}')
print(f'col_hasna_object_predict : {col_hasna_object_predict}')
print(f'col_gen_flagfields : {col_gen_flagfields}')
print(f'col_gen_datefields : {col_gen_datefields}')

In [ ]:
# except train features
dropfeatures = ['SalePrice','Id']
print(f'train data need to drop Features : {dropfeatures}')

## 3.2 타깃 변수 분포 확인하기

In [ ]:
def fig_distplot(fig, data, measure):
    (mu, sigma) = norm.fit(data)
    # distplot 으로 타깃 변수의 분포를 확인합니다. kde = True를 입력하면 커널밀도추정 결과를 함께 표기할 수 있습니다.
    sns.distplot(data, kde=True, fit=norm)
    fig.legend(['KDE', f'N({round(mu, 2)},{round(sigma**2, 2)})'], loc='upper right')
    fig.set_title(f'{measure} Distribution', loc='center')
    fig.set_xlabel(f'{measure}')

def fig_probplot(fig, data, measure):
    # 정규분포와 타깃변수의 분포를 비교합니다.
    res = probplot(data, plot=fig)
    fig.set_title(f'{measure} Probability Plot', loc='center')

def QQ_plot(data, measure, logplot=False):
    if logplot :
        # 그래프의 크기를 지정합니다.
        fig = plt.figure(figsize=(10,8))
        # 2행 2열의 첫번째 히스토그램 그래프입니다.
        fig1 = fig.add_subplot(221)
        fig_distplot(fig1, data, measure)       
        # 2행 2열의 두번째 QQ Plot 그래프입니다.
        fig2 = fig.add_subplot(222)
        fig_probplot(fig2, data, measure)
        # 비정규화된 데이타의 학습 품질을 높이는 방법으로 원 데이타에 Log 함수를 취하여 정규분포처리로 환원하여 학습하기.
        # Train 시 Log 함수 처리 : np.log1p(a),  Log 함수 결과값을 원래의 값으로 환원하기 : np.expm1(b)
        # 2행 2열의 np.log1p 세번째 히스토그램 그래프입니다.
        fig3 = fig.add_subplot(223)
        fig_distplot(fig3, np.log1p(data), f'np.log1p {measure}')
        # 2행 2열의 np.log1p 네번째 QQ Plot 그래프입니다.
        fig4 = fig.add_subplot(224)
        fig_probplot(fig4, np.log1p(data), f'np.log1p {measure}')
    else:
        # 그래프의 크기를 지정합니다.
        fig = plt.figure(figsize=(10,4))
        # 1행 2열의 첫번째 히스토그램 그래프입니다.
        fig1 = fig.add_subplot(121)
        fig_distplot(fig1, data, measure)       
        # 1행 2열의 두번째 QQ Plot 그래프입니다.
        fig2 = fig.add_subplot(122)
        fig_probplot(fig2, data, measure)

    plt.tight_layout()
    plt.show()

In [ ]:
QQ_plot(df_eda['SalePrice'], 'SalePrice', True)

In [ ]:
cond1 = (np.log1p(df_eda['SalePrice']) >= 10.7)
cond2 = (np.log1p(df_eda['SalePrice']) <  13.5)
df_eda_new = df_eda.loc[cond1]
print(df_eda.shape, df_eda_new.shape)
df_eda_new = df_eda.loc[cond2]
print(df_eda.shape, df_eda_new.shape)
df_eda_new = df_eda.loc[cond1 & cond2]
print(df_eda.shape, df_eda_new.shape)
QQ_plot(np.log1p(df_eda_new['SalePrice']), 'SalePrice', False)

In [ ]:
cond1 = (np.log1p(df_eda['SalePrice']) >= 10.7)
cond2 = (np.log1p(df_eda['SalePrice']) <  13.3)
df_eda_new = df_eda.loc[cond1]
print(df_eda.shape, df_eda_new.shape)
df_eda_new = df_eda.loc[cond2]
print(df_eda.shape, df_eda_new.shape)
df_eda_new = df_eda.loc[cond1 & cond2]
print(df_eda.shape, df_eda_new.shape)
QQ_plot(np.log1p(df_eda_new['SalePrice']), 'SalePrice', False)

## 3.3. Features와 Target 값 상관관계 분석

In [ ]:
print(f'col_nona_numerical : {col_nona_numerical}')
print(f'col_nona_numerical_predict : {col_nona_numerical_predict}')
print(f'col_hasna_numerical : {col_hasna_numerical}')
print(f'col_hasna_numerical_predict : {col_hasna_numerical_predict}')
print(f'col_nona_object : {col_nona_object}')
print(f'col_nona_object_predict : {col_nona_object_predict}')
print(f'col_hasna_object : {col_hasna_object}')
print(f'col_hasna_object_predict : {col_hasna_object_predict}')
print(f'col_gen_flagfields : {col_gen_flagfields}')
print(f'col_gen_datefields : {col_gen_datefields}')
# except train features
dropfeatures = ['SalePrice','Id']
print(f'train data need to drop Features : {dropfeatures}')

In [ ]:
def multiplots(col, target, data, huecol=''):
    
    # 각 행이 1개이고, 열이 5개인 서브 플롯을 생성합니다.
    fig, axes = plt.subplots(nrows=1, ncols=5, sharey=True, figsize=(28,4))
    #fig, axes = plt.subplots(nrows=2, ncols=4, sharey=True, sharex=True, figsize=(28,4))
    #sns.despine(left=True)
    
    # 서브 플롯 간의 세로 간격(hspace) 가로 간격(wspace)을 설정합니다. 
    fig.subplots_adjust(hspace=.4, wspace=.1)

    #countplot, barplot, histogram, pointplot
    #scatter, regplot, boxplot

    charts = [
              {'Index' : 0, 'ChartName' : 'LinePlot'   , 'Chart' : 'sns.lineplot'    , 'Param' : 'x,y,d'},
              {'Index' : 1, 'ChartName' : 'PointPlot'  , 'Chart' : 'sns.pointplot'   , 'Param' : 'x,y,d'},
              {'Index' : 2, 'ChartName' : 'ScatterPlot', 'Chart' : 'sns.scatterplot' , 'Param' : 'x,y,d'},
#               {'Index' : 2, 'ChartName' : 'ViolinPlot' , 'Chart' : 'sns.violinplot'  , 'Param' : 'x,y,d'},
              {'Index' : 3, 'ChartName' : 'RegPlot'    , 'Chart' : 'sns.regplot'     , 'Param' : 'x,y,d'},
              {'Index' : 4, 'ChartName' : 'BoxPlot'    , 'Chart' : 'sns.boxplot'     , 'Param' : 'x,y,d'},
             ]
    for idx, ax in enumerate(axes.flatten()):
        isvisible = False
        for chart in charts:
            if idx == chart['Index']:
                # print(idx, chart["ChartName"], chart['Param'], ax)
                func = eval(chart['Chart'])
                if chart['Param'] == 'x,y,d':
                    func(x=col, y=target, data=data, ax=ax)
                elif chart['Param'] == 'x,y,h,d':
                    func(x=col, y=target, hue=huecol, data=data, ax=ax)
                elif chart['Param'] == 'y,x,d':
                    func(y=col, x=target, data=data, ax=ax)
                elif chart['Param'] == 'y,x,h,d':
                    func(y=col, x=target, hue=huecol, data=data, ax=ax)
                ax.set_title(f'{col} - {chart["ChartName"]}')
                ax.set_xlabel('')
                ax.set_ylabel('')
                isvisible = True
        ax.set_visible(True if isvisible else False)

    plt.tight_layout() # 서브 플롯 layout 조정

    # 각 행이 1개이고, 열이 5개인 서브 플롯을 생성합니다.
#     fig2, axes2 = plt.subplots(nrows=1, ncols=5, sharey=True, figsize=(28,4))
    fig2, axes2 = plt.subplots(nrows=1, ncols=5, figsize=(28,4))
    #sns.despine(left=True)
    
    # 서브 플롯 간의 세로 간격(hspace) 가로 간격(wspace)을 설정합니다. 
    fig2.subplots_adjust(hspace=.4, wspace=.1)
    
#     print(axes2)
# #     sns.displot(data=data, x=col, kind='kde', ax=axes2[0])
# #     sns.displot(data=data, x=col, kde=True, ax=axes2[1])
#     sns.kdeplot(data=data, x=col, ax=axes2[0])
#     sns.distplot(data[col],ax=axes2[1])
#     sns.histplot(data=data, x=col, ax=axes2[2])
# #     sns.histplot(data=data, x=col, ax=axes2[1])

    charts2 = [
              {'Index' : 0, 'ChartName' : 'KdePlot'     , 'Chart' : 'sns.kdeplot'     , 'Param' : 'd,x'},
              {'Index' : 1, 'ChartName' : 'DistPlot'    , 'Chart' : 'sns.distplot'    , 'Param' : 'dc'},
              {'Index' : 2, 'ChartName' : 'HistPlot'    , 'Chart' : 'sns.histplot'    , 'Param' : 'd,x'},
             ]
    for idx, ax in enumerate(axes2.flatten()):
        isvisible = False
        for chart in charts2:
            if idx == chart['Index']:
                # print(idx, chart["ChartName"], chart['Param'], ax)
                func = eval(chart['Chart'])
                if chart['Param'] == 'dc':
                    func(data[col], ax=ax)
                elif chart['Param'] == 'd,x':
                    func(data=data, x=col, ax=ax)
                elif chart['Param'] == 'd,x,h':
                    func(data=data, x=col, hue=huecol, ax=ax)
                ax.set_title(f'{col} - {chart["ChartName"]}')
                ax.set_xlabel('')
                ax.set_ylabel('')
                isvisible = True
        ax.set_visible(True if isvisible else False)

    plt.tight_layout()  # 서브 플롯 layout 조정
    plt.show()

In [ ]:
# Search Columns
searchcols = col_nona_numerical + col_hasna_numerical + col_nona_object + col_hasna_object + col_gen_flagfields + col_gen_datefields
for dropfeature in dropfeatures:
    searchcols.remove(dropfeature)

In [ ]:
# Draw Chart
for idx, feature in enumerate(searchcols):
    print(f'.... index : {idx} ... col : [{feature}] huecol : []')
#     multiplots(feature, 'SalePrice', df_eda, '')

## 3.4. Feature간 correlation 높은 것 그룹핑

In [ ]:
def df_fcorr(df, corr_value = 0.9):
    ''' Numerical Features간 correlation이 있는 필드 정보 리스트 추출
    '''
    df_corr = df.corr()
    indexes = df_corr.index

    corr_features_list = []
    for idx in range(0, math.ceil(len(indexes)/2)):
        if indexes[idx] == 'Id' :
            continue
#         print(f'index : {idx} = {indexes[idx]}')
        high_corr = corr_value
        cond1 = df_corr[indexes[idx]] < 1
        cond2 = df_corr[indexes[idx]] >= high_corr
        if df_corr[(cond1) & (cond2)].shape[0] > 0 :
            corr_features = set(df_corr[(cond1) & (cond2)].index)
            corr_features.add(indexes[idx])
#             print(corr_features)
            if not corr_features in corr_features_list :
                corr_features_list.append(corr_features)
    return corr_features_list
corr_features = df_fcorr(df_eda, 0.8)
print(corr_features)

In [ ]:
# Draw Chart
for idx, feature in enumerate(corr_features):
    print(f'.... index : {idx} ... col : [{feature}] huecol : []')
#     multiplots(list(feature)[0], list(feature)[1], df_eda, '')

## 3.5. Features와 Target 값 상관관계 및 Feature간 correlation 높은 것 그룹핑 분석결과

### 3.5.1 분석 제외 컬럼 목록

In [ ]:
# exception feature list
col_except_feature_list = []
for idx, feature in enumerate(searchcols):
    if idx in [31,32,39,45,64,86,95]:
        print(f'exception feature : index : {idx} [{feature}]')
        col_except_feature_list.append(feature)
print(col_except_feature_list)

### 3.5.2 Outlier 처리 대상 컬럼 목록

In [ ]:
# outlier feature list
col_need_outlier_handing_list = []
for idx, feature in enumerate(searchcols):
    if idx in [1,4,6,7,8,9,10,11,12,13,23,24,25,26,30,33,34]:
        print(f'need outlier heanding feature : index : {idx} [{feature}]')
        col_need_outlier_handing_list.append(feature)
print(col_need_outlier_handing_list)

In [ ]:
#Outlier 처리 
col_result_outlier_list = [] # outlier 처리 결과 features
for col in col_need_outlier_handing_list: # 상관관계 분석결과 need outlier handing features
    p01 = df_eda[col].quantile(0.01) 
    p99 = df_eda[col].quantile(0.99) 

    df_eda[f'gen_outlier_{col}']   = df_eda[col].clip(p01, p99) # assign Train quantile value
    col_result_outlier_list.append(f'gen_outlier_{col}')

In [ ]:
def compare_multiplots(befcol, aftcol, target, data, huecol='') :
    
    # 각 행이 1개이고, 열이 6개인 서브 플롯을 생성합니다.
#     fig, axes = plt.subplots(nrows=1, ncols=6, sharey=True, figsize=(28,4))
    fig, axes = plt.subplots(nrows=1, ncols=6, figsize=(28,4))
    
    # 서브 플롯 간의 세로 간격(hspace) 가로 간격(wspace)을 설정합니다. 
    fig.subplots_adjust(hspace=.4, wspace=.1)

    charts = [
              {'Index' : 0, 'ChartName' : 'LinePlot'   , 'Chart' : 'sns.lineplot'    , 'Param' : 'bx,y,d'},
              {'Index' : 1, 'ChartName' : 'LinePlot'   , 'Chart' : 'sns.lineplot'    , 'Param' : 'ax,y,d'},
              {'Index' : 2, 'ChartName' : 'RegPlot'    , 'Chart' : 'sns.regplot'     , 'Param' : 'bx,y,d'},
              {'Index' : 3, 'ChartName' : 'RegPlot'    , 'Chart' : 'sns.regplot'     , 'Param' : 'ax,y,d'},
              {'Index' : 4, 'ChartName' : 'DistPlot'   , 'Chart' : 'sns.distplot'    , 'Param' : 'dbc'},
              {'Index' : 5, 'ChartName' : 'DistPlot'   , 'Chart' : 'sns.distplot'    , 'Param' : 'dac'},
             ]
    for idx, ax in enumerate(axes.flatten()):
        isvisible = False
        for chart in charts:
            if idx == chart['Index']:
                # print(idx, chart["ChartName"], chart['Param'], ax)
                func = eval(chart['Chart'])
                if chart['Param'] == 'bx,y,d':
                    func(x=befcol, y=target, data=data, ax=ax)
                    ax.set_title(f'{befcol} - {chart["ChartName"]}')
                elif chart['Param'] == 'bx,y,h,d':
                    func(x=befcol, y=target, hue=huecol, data=data, ax=ax)
                    ax.set_title(f'{befcol} - {chart["ChartName"]}')
                elif chart['Param'] == 'ax,y,d':
                    func(x=aftcol, y=target, data=data, ax=ax)
                    ax.set_title(f'{aftcol} - {chart["ChartName"]}')
                elif chart['Param'] == 'ax,y,h,d':
                    func(x=aftcol, y=target, data=data, ax=ax)
                    ax.set_title(f'{aftcol} - {chart["ChartName"]}')
                elif chart['Param'] == 'dbc':
                    func(data[befcol], ax=ax)
                    ax.set_title(f'{befcol} - {chart["ChartName"]}')
                elif chart['Param'] == 'dac':
                    func(data[aftcol], ax=ax)
                    ax.set_title(f'{aftcol} - {chart["ChartName"]}')
                ax.set_xlabel('')
                ax.set_ylabel('')
                isvisible = True
        ax.set_visible(True if isvisible else False)

    plt.tight_layout() # 서브 플롯 layout 조정
    plt.show()

In [ ]:
# Outlier 처리 후 Draw Chart
for idx in range(0, len(col_need_outlier_handing_list)):
    print(f'... idx [{idx}] ... total [{len(col_need_outlier_handing_list)}]')
#     compare_multiplots(col_need_outlier_handing_list[idx], col_result_outlier_list[idx], 'SalePrice', df_eda, '')

### 3.5.3 correlation 높은 그룹 선별 목록

In [ ]:
# correlation high feature list
col_corr_high_list = []
for idx, feature in enumerate(corr_features):
    if idx in [3,4,5]:
        print(f'correlation high feature : index : {idx} [{feature}]')
        col_corr_high_list.append(feature)
print(col_corr_high_list)
print(corr_features)

## 3-77 df_eda 제거하기

In [ ]:
del df_eda

# 4. 데이터 전처리(Data Preprecessing)

## 4.1 전처리용 데이터 준비하기

In [ ]:
# 모델 성능 그래프 Label 정보 활용 
trynum = 0

In [ ]:
# 그래프 초기화
utils.clear_error()


In [ ]:
# 전처리를 위한 학습용 데이터와 평가용 데이터를 복사합니다.
df_prep_train = train.copy()
df_prep_predict = test.copy()

In [ ]:
# 작업 명세서 초기화
change_work = ''

## 4.2 예측값 지수함수처리

In [ ]:
print(df_prep_train['SalePrice'].shape, df_prep_train['SalePrice'].head())
df_prep_train['SalePrice'] = df_prep_train['SalePrice'].apply(lambda x : np.log1p(x))
cond1 = (df_prep_train['SalePrice'] >= 10.7)
cond2 = (df_prep_train['SalePrice'] <  13.5)
df_prep_train = df_prep_train.loc[cond1 & cond2]
print(df_prep_train['SalePrice'].shape, df_prep_train['SalePrice'].head())
change_work += 'Filter 10.7 <= SalePrice <13.5,'
# change_work += 'SalePrice Filter 10.7<= .. <13.3 '
# change_work += 'SalePrice NoFilter '

## 4.3 결측치 보정 전 신규 Feature 생성

#### 1) Train Data 처리

In [ ]:
change_work += 'add flag field in train,'
# flag field 추가
add_flagfields(df_prep_train)
# 추가 flag field 변수
col_gen_flagfields = list(df_prep_train.filter(regex='gen_existflag_').columns)
df_prep_train[col_gen_flagfields].describe()

#### 2) Predict Data 처리

In [ ]:
change_work += 'add flag field in predict,'
add_flagfields(df_prep_predict)
# 추가 flag field 변수
df_prep_predict[col_gen_flagfields].describe()

## 4.4 결측치 보정

### 4.4.1 Numerical 컬럼 결측치 처리

#### 1) Train Data 처리

In [ ]:
change_work += 'numeric fillna in train,'
# numerical feature 결측치 보정하기
df_numerical_fillna(df_prep_train, 'Train')

#### 2) Predict Data 처리

In [ ]:
change_work += 'numeric fillna in predict,'
# numerical feature 결측치 보정하기
df_numerical_fillna(df_prep_predict, 'Predict')

### 4.4.2 Object 컬럼 결측치 처리 및 숫자형 변환 동시 처리

#### 1) Train Data 처리

In [ ]:
change_work += 'object fillna and convert to num in train,'
# object feature 결측치 보정 및 숫자형 변환
df_object_fillna(df_prep_train, df_prep_predict, 'Train')

#### 2) Predict Data 처리

In [ ]:
change_work += 'object fillna and convert to num in predict,'
# object feature 결측치 보정 및 숫자형 변환
df_object_fillna(df_prep_predict, df_prep_train, 'Predict')

## 4.5 결측치 보정 후 신규 Feature 추가

#### 1) Train Data 처리

In [ ]:
change_work += 'add datefields in train,'
# 추가 date field 생성
add_datafields(df_prep_train) 
# 추가 date field 변수
col_gen_datefields = list(df_prep_train.filter(regex='gen_grpdate_').columns)
print(col_gen_datefields)

#### 2) Predict Data 처리

In [ ]:
change_work += 'add datefields in predict,'
# 추가 date field 생성
add_datafields(df_prep_predict) 

## 4.6 Skewed Data log function 처리

In [ ]:
## result message
change_work += 'skewed data np.log1p function before outliyer process '
for col in col_union_skewed_0point5 :
    bef_data = df_prep_train[col]
    df_prep_train[col] = np.log1p(df_prep_train[col])
    df_prep_predict[col] = np.log1p(df_prep_predict[col])
#     print(f'skewed feature np.log1p : {col} - bef [{bef_data}], aft [{df_prep_train[col]}]')
    print(f'skewed feature np.log1p : {col}')

# change_work += 'skewed data boxcox1p lambda(0.15) function before outliyer process '
# lambda_select = 0.1  # 0.15
# for col in col_union_skewed_0point5 :
#     bef_data = df_prep_train[col]
#     df_prep_train[col] = boxcox1p(df_prep_train[col], lambda_select)
#     df_prep_predict[col] = boxcox1p(df_prep_predict[col], lambda_select)
# #     print(f'skewed feature boxcox1p : {col} - bef [{bef_data}], aft [{df_prep_train[col]}]')
#     print(f'skewed feature boxcox1p : {col}')

## 4.7 Outlier 

#### 1) Train Data 처리

In [ ]:
#Outlier flag 설정  
# def outlier(x, p01, p99):
#   if x < p01 or x > p99:
#     return 'Y'
#   else:
#     return 'N'
# for col in col_need_outlier_handing_list: # 상관관계 분석결과 need outlier handing features
#     p01 = df_prep_train[col].quantile(0.01) 
#     p99 = df_prep_train[col].quantile(0.99) 

#     df_prep_train[f'gen_outlier_flag_{col}'] = df_prep_train[col].apply(outlier, args=(p01, p99))  # check info

In [ ]:
change_work += 'outlier in train,'
#Outlier 처리 
for col in col_need_outlier_handing_list: # 상관관계 분석결과 need outlier handing features
    p01 = df_prep_train[col].quantile(0.01) 
    p99 = df_prep_train[col].quantile(0.99) 

    df_prep_train[f'gen_outlier_{col}']   = df_prep_train[col].clip(p01, p99) # assign Train quantile value

#### 2) Predict Data 처리

In [ ]:
change_work += 'outlier in predict,'
#Outlier 처리 
for col in col_need_outlier_handing_list: # 상관관계 분석결과 need outlier handing features
    p01 = df_prep_train[col].quantile(0.01) # Train quantile value
    p99 = df_prep_train[col].quantile(0.99) # Train quantile value

    df_prep_predict[f'gen_outlier_{col}'] = df_prep_predict[col].clip(p01, p99) # assign Train quantile value
#     print(f'p01 : {p01}, p99: {p99}')
#     display(df_prep_predict[col].clip(p01, p99))

## 4.8 데이타 표준화 처리

In [ ]:
scaler = StandardScaler()

In [ ]:
# print(f'col_nona_numerical : {col_nona_numerical}')
# print(f'col_nona_numerical_predict : {col_nona_numerical_predict}')
# print(f'col_hasna_numerical : {col_hasna_numerical}')
# print(f'col_hasna_numerical_predict : {col_hasna_numerical_predict}')
# print(f'col_nona_object : {col_nona_object}')
# print(f'col_nona_object_predict : {col_nona_object_predict}')
# print(f'col_hasna_object : {col_hasna_object}')
# print(f'col_hasna_object_predict : {col_hasna_object_predict}')
# print(f'col_gen_flagfields : {col_gen_flagfields}')
# print(f'col_gen_datefields : {col_gen_datefields}')
# print(f'col_except_feature_list : {col_except_feature_list}')
# print(f'col_need_outlier_handing_list : {col_need_outlier_handing_list}')
# print(f'col_result_outlier_list : {col_result_outlier_list}')

# dropfeatures = ['SalePrice','Id']
# print(f'train data need to drop Features : {dropfeatures}')
# 학습 데이터 각 변수의 평균값과 표준편차를 저장한 후, 이를 사용하여 학습 및 평가용 데이터를 변환
apply_column = col_nona_numerical + col_hasna_numerical + col_nona_object + col_hasna_object + col_gen_datefields \
               + col_result_outlier_list  # outlier generate feature 추가
for dropfeature in dropfeatures: #  index와 predict feature 제외
    if dropfeature in dropfeatures:
        apply_column.remove(dropfeature)
for exceptfeature in col_except_feature_list: # 상관관계 분석 결과 except feature list 제외
    if exceptfeature in apply_column:
        apply_column.remove(exceptfeature)
for outlier in col_need_outlier_handing_list: # need outlier handing feature 제외
    if outlier in apply_column:
        apply_column.remove(outlier)
change_work += 'stadardscaler in train,'
# Train data standardscale
df_prep_train[apply_column] = scaler.fit_transform(df_prep_train[apply_column])
change_work += 'stadardscaler in predict,'
# Predict data standardscale
df_prep_predict[apply_column] = scaler.fit_transform(df_prep_predict[apply_column])

In [ ]:
sp.mean(df_prep_train[apply_column], axis=0).head(3).round(3), sp.std(df_prep_predict[apply_column], axis=0).head(3).round(3)

### 4.8.1 표준화 처리 데이터와 'SalePrice' 상관관계가 높은 컬럼만 선별하기

In [ ]:
details = rstr(df_prep_train, 'SalePrice')
cond1 = details['corr SalePrice'] >= 0.1
cond2 = details['corr SalePrice'] <= -0.1
details_corr = details[(cond1) | (cond2)]
details_corr = details                ## Ridge도 전체 컬럼으로 분석하는 것이 성능 좋음으로 모든 컬럼으로 적용함.
select_cols = set(details_corr.index)
print(select_cols, len(select_cols))

In [ ]:
for exceptfeature in col_except_feature_list: # 상관관계 분석 결과 except feature list 제외
    if exceptfeature in select_cols :
        select_cols.remove(exceptfeature)
for outlierfeature in col_need_outlier_handing_list: # outlier 필요 대상 list는 제외하고, 대신 처리 결과 리스트를 추가하기
    if outlierfeature in select_cols :
        select_cols.remove(outlierfeature)
for dropfeature in dropfeatures: #  index와 predict feature 제외
    if dropfeature in select_cols :
        select_cols.remove(dropfeature)

In [ ]:
# Feature간 유사도가 높은 컬럼들은 1개만 select_cols에 포함되도록 처리
for items in col_corr_high_list:
    is_exist = False
    for item in list(items):
        if item in select_cols :
            if is_exist == False :
                is_exist = True
            else :
                select_cols.remove(item)

len(select_cols)

In [ ]:
# result message
# change_work += '유사컬럼은 1개만 선택,''
# change_work += '컬럼선별:corr >= |0.1|,'

# 5. 모델링(Modeling)

## 5.1 모델 성능 평가함수 정의

In [ ]:
# mean_squared_error 의 매개변수 squared 가 False 이면 RMSE 를 반환합니다.
def rmse(y_true, y_pred):
  return round(mean_squared_error(y_true, y_pred, squared=False), 4)

## 5.2 모델 공통 정보 설정

In [ ]:
MODEL_SEED = 123

In [ ]:
TEST_SIZE = 0.15
# result message
change_work += 'SEED=123,TEST_SIZE=0.15,'
# change_work += 'SEED=123,TEST_SIZE=0.1,'

## 5.3 모델구축

### 5.4.1 Ridge 회귀모델

#### 5.4.1.1 데이타 분할

In [ ]:
# result message
# change_work += 'Ridge : 컬럼선별:corr >= |0.5|,'
# change_work += 'Ridge : ALL,'
change_work += 'Ridge : exceptfeature, outlierfeature remove, '

# 모델에 입력으로 사용될 최종 컬럼을 정의합니다.
col_X = col_nona_numerical + col_hasna_numerical + col_nona_object + col_hasna_object + col_gen_flagfields + col_gen_datefields \
        + col_result_outlier_list # outlier 필요 대상 list는 제외하고, 대신 처리 결가 리스트를 추가하기
for exceptfeature in col_except_feature_list: # 상관관계 분석 결과 except feature list 제외
    if exceptfeature in col_X :
        col_X.remove(exceptfeature)
for outlierfeature in col_need_outlier_handing_list: # outlier 필요 대상 list는 제외하고, 대신 처리 결가 리스트를 추가하기
    if outlierfeature in col_X :
        col_X.remove(outlierfeature)
for dropfeature in dropfeatures: #  index와 predict feature 제외
    if dropfeature in col_X :
        col_X.remove(dropfeature)
# 모델에 입력으로 사용될 최종 컬럼을 정의합니다.
# col_X = list(select_cols)
col_y = 'SalePrice'

print(col_X)
print(f'Ridge Feature 수 : { len(col_X) }')

# 학습용 데이터를 정의합니다.
x_ridge_train_raw = df_prep_train[col_X]
y_ridge_train_raw = df_prep_train[col_y]

# 평가용 데이터를 정의합니다.
x_ridge_predict = df_prep_predict[col_X]

# 학습 데이터 중 일부를 검증 데이터 셋으로 분할합니다. 
SEED = 71
x_ridge_train, x_ridge_valid, y_ridge_train, y_ridge_valid = train_test_split(x_ridge_train_raw, y_ridge_train_raw, test_size=TEST_SIZE, shuffle=True, random_state=SEED)

#### 5.4.1.2 Ridge 회귀모델

In [ ]:
%%time

# change_work += 'ridge  range(170, 210, 5)'
change_work += ' ridge np.logspace(-2,1,10)'

# log 단위(1e+01)로 1.e-02 ~ 1.e+01 구간에 대해 매개변수를 탐색합니다.
ridge_params = {
    'alpha': np.logspace(-2,1,10),
#     'alpha': range(170, 210, 5),
    'random_state' : [MODEL_SEED]
}


In [ ]:
%%time

# GridSearchCV를 이용하여 Ridge 모델을 저장합니다.
ridge_model = GridSearchCV(Ridge(), param_grid=ridge_params, refit=True, scoring='neg_root_mean_squared_error')
ridge_model.fit(x_ridge_train, y_ridge_train)

In [ ]:
col_cv_results = ['params', 'mean_test_score', 'rank_test_score']
pd.DataFrame(ridge_model.cv_results_)[col_cv_results]

In [ ]:
# pd.DataFrame(ridge_model.cv_results_)

In [ ]:
print(ridge_model.best_params_)

### 5.4.2 Random Forest 모델

#### 5.4.2.1 데이타 분할

In [ ]:
# result message
# change_work += 'Rf : All,'
# change_work += 'Rf : outlierfeature remove, '
change_work += 'Rf : exceptfeature, outlierfeature remove, '

# 모델에 입력으로 사용될 최종 컬럼을 정의합니다.
col_X = col_nona_numerical + col_hasna_numerical + col_nona_object + col_hasna_object + col_gen_flagfields + col_gen_datefields \
        + col_result_outlier_list # outlier 처리 결과 리스트를 추가하기
for exceptfeature in col_except_feature_list: # 상관관계 분석 결과 except feature list 제외
    if exceptfeature in col_X :
        col_X.remove(exceptfeature)
for outlierfeature in col_need_outlier_handing_list: # outlier 필요 대상 list는 제외하고, 대신 처리 결가 리스트를 추가하기
    if outlierfeature in col_X :
        col_X.remove(outlierfeature)
for dropfeature in dropfeatures: #  index와 predict feature 제외
    if dropfeature in col_X :
        col_X.remove(dropfeature)
col_y = 'SalePrice'

print(col_X)
print(f'RF Feature 수 : { len(col_X) }')

# 학습용 데이터를 정의합니다.
x_rf_train_raw = df_prep_train[col_X]
y_rf_train_raw = df_prep_train[col_y]

# 평가용 데이터를 정의합니다.
x_rf_predict = df_prep_predict[col_X]

# 학습 데이터 중 일부를 검증 데이터 셋으로 분할합니다. 
SEED = 71
x_rf_train, x_rf_valid, y_rf_train, y_rf_valid = train_test_split(x_rf_train_raw, y_rf_train_raw, test_size=TEST_SIZE, shuffle=True, random_state=SEED)

#### 5.4.2.2 Random Forest 모델

In [ ]:
%%time

change_work += 'Rf : param : [1300],[8] '

rf_params = {'n_estimators': [1300],
             'min_samples_split' : [2],
             'min_samples_leaf' : [1],
             'max_features' : [8],
             'random_state' : [MODEL_SEED]
} 

# GridSearchCV를 이용하여 Randomforest 모델을 저장합니다.
rf_model = GridSearchCV(RandomForestRegressor(), param_grid=rf_params, scoring='neg_root_mean_squared_error')
rf_model.fit(x_rf_train, y_rf_train)

In [ ]:
pd.DataFrame(rf_model.cv_results_)[col_cv_results]

In [ ]:
# pd.DataFrame(rf_model.cv_results_)

In [ ]:
rf_model.best_params_

### 5.4.3 XGBoost 모델

#### 5.4.3.1 데이타 분할

In [ ]:
# result message
# change_work += 'XGB : All,'
# change_work += 'XGB : outlierfeature remove, '
change_work += 'XGB : exceptfeature, outlierfeature remove, '

# 모델에 입력으로 사용될 최종 컬럼을 정의합니다.
col_X = col_nona_numerical + col_hasna_numerical + col_nona_object + col_hasna_object + col_gen_flagfields + col_gen_datefields \
        + col_result_outlier_list # outlier 처리 결과 리스트를 추가하기
for exceptfeature in col_except_feature_list: # 상관관계 분석 결과 except feature list 제외
    if exceptfeature in col_X :
        col_X.remove(exceptfeature)
for outlierfeature in col_need_outlier_handing_list: # outlier 필요 대상 list는 제외하고, 대신 처리 결가 리스트를 추가하기
    if outlierfeature in col_X :
        col_X.remove(outlierfeature)
for dropfeature in dropfeatures: #  index와 predict feature 제외
    if dropfeature in col_X :
        col_X.remove(dropfeature)
col_y = 'SalePrice'


print(col_X)
print(f'XGB Feature 수 : { len(col_X) }')

# 학습용 데이터를 정의합니다.
x_xgb_train_raw = df_prep_train[col_X]
y_xgb_train_raw = df_prep_train[col_y]

# 평가용 데이터를 정의합니다.
x_xgb_predict = df_prep_predict[col_X]

# 학습 데이터 중 일부를 검증 데이터 셋으로 분할합니다. 
SEED = 71
x_xgb_train, x_xgb_valid, y_xgb_train, y_xgb_valid = train_test_split(x_xgb_train_raw, y_xgb_train_raw, test_size=TEST_SIZE, shuffle=True, random_state=SEED)

#### 5.4.3.2 XGBoost 모델

In [ ]:
# change_work += 'XGB : param : [400, 410],[0.05,0.1]  '
# change_work += 'XGB : param : [348, 349, 350],[0.1]  ' 
change_work += 'XGB : param : [350, 360, 370],[0.1]  ' 

# XGBoost 모델
xgb_params = {
    'n_estimators': [350, 360, 370],  # 100 1000, 500
    'learning_rate' : [0.1], # default : 0.3 (0-1)
    'eval_metric': ['rmse'],
 #   'max_depth' : [4, 6, 8, 9], # default : 6
    'objective': ['reg:squarederror'] ,
    'random_state' : [MODEL_SEED]
} 

In [ ]:
%%time

# GridSearchCV를 이용하여 XGBoost 모델을 저장합니다.
xgb_model = GridSearchCV(XGBRegressor(), param_grid=xgb_params, cv=5, scoring='neg_root_mean_squared_error')
xgb_model.fit(x_xgb_train, y_xgb_train)

In [ ]:
pd.DataFrame(xgb_model.cv_results_)[col_cv_results]

In [ ]:
# pd.DataFrame(xgb_model.cv_results_)

In [ ]:
xgb_model.best_params_

### 5.4.4 Stack 모델

#### 5.4.4.1 데이타 분할

In [ ]:
# result message
# change_work += 'Stack : All,'
# change_work += 'Stack : outlierfeature remove, '
change_work += 'Stack : exceptfeature, outlierfeature remove, '

# 모델에 입력으로 사용될 최종 컬럼을 정의합니다.
col_X = col_nona_numerical + col_hasna_numerical + col_nona_object + col_hasna_object + col_gen_flagfields + col_gen_datefields \
        + col_result_outlier_list # outlier 처리 결과 리스트를 추가하기
for exceptfeature in col_except_feature_list: # 상관관계 분석 결과 except feature list 제외
    if exceptfeature in col_X :
        col_X.remove(exceptfeature)
for outlierfeature in col_need_outlier_handing_list: # outlier 필요 대상 list는 제외하고, 대신 처리 결가 리스트를 추가하기
    if outlierfeature in col_X :
        col_X.remove(outlierfeature)
for dropfeature in dropfeatures: #  index와 predict feature 제외
    if dropfeature in col_X :
        col_X.remove(dropfeature)
col_y = 'SalePrice'


print(col_X)
print(f'Stack Feature 수 : { len(col_X) }')

# 학습용 데이터를 정의합니다.
x_stack_train_raw = df_prep_train[col_X]
y_stack_train_raw = df_prep_train[col_y]

# 평가용 데이터를 정의합니다.
x_stack_predict = df_prep_predict[col_X]

# 학습 데이터 중 일부를 검증 데이터 셋으로 분할합니다. 
SEED = 71
x_stack_train, x_stack_valid, y_stack_train, y_stack_valid = train_test_split(x_stack_train_raw, y_stack_train_raw, test_size=TEST_SIZE, shuffle=True, random_state=SEED)

#### 5.4.4.2 stack 모델

In [ ]:
%%time

# Stack up all the models above, optimized using xgboost
stack = StackingCVRegressor(regressors=(xgb_model, ridge_model, rf_model),
                                meta_regressor=xgb_model,
                                use_features_in_secondary=True)
stack.fit(np.array(x_stack_train), np.array(y_stack_train))

### 5.5 최종모델 구축

In [ ]:
# 최적의 하이퍼파라미터로 학습된 릿지 회귀 모델을 저장합니다. 
best_ridge = ridge_model.best_estimator_

# 최적의 하이퍼파라미터로 학습된 랜덤포레스트 모델을 저장합니다. 
best_rf = rf_model.best_estimator_

# 최적의 하이퍼파라미터로 학습된 XGBoost 모델을 저장합니다.  
best_xgb = xgb_model.best_estimator_

# 학습용 데이터셋의 타깃 변수 예측 
y_pred_tr_ridge = best_ridge.predict(x_ridge_train)
y_pred_tr_rf = best_rf.predict(x_rf_train)
y_pred_tr_xgb = best_xgb.predict(x_xgb_train)

# 검증용 데이터셋의 타깃 변수 예측 
y_pred_va_ridge = best_ridge.predict(x_ridge_valid)
y_pred_va_rf = best_rf.predict(x_rf_valid)
y_pred_va_xgb = best_xgb.predict(x_xgb_valid)

# 평가용 데이터셋의 타깃 변수 예측 
y_pred_te_ridge = best_ridge.predict(x_ridge_predict)
y_pred_te_rf = best_rf.predict(x_rf_predict)
y_pred_te_xgb = best_xgb.predict(x_xgb_predict)

# 앙상블 모형의 학습 및 검증 데이터에 대한 타깃 변수 예측 
y_pred_tr_en1= np.mean([y_pred_tr_ridge, y_pred_tr_xgb], axis=0)
y_pred_va_en1= np.mean([y_pred_va_ridge, y_pred_va_xgb], axis=0)
y_pred_te_en1= np.mean([y_pred_te_ridge, y_pred_te_xgb], axis=0)
# y_pred_tr_en1= np.mean([y_pred_tr_ridge, y_pred_tr_rf], axis=0)
# y_pred_va_en1= np.mean([y_pred_va_ridge, y_pred_va_rf], axis=0)
# y_pred_te_en1= np.mean([y_pred_te_ridge, y_pred_te_rf], axis=0)

print(f'en  : y_pred_tr_ridge, y_pred_tr_xgb')
print(f'en1 : y_pred_tr_ridge, y_pred_tr_rf')

# ensenble 모델 처리 함수  : 0.5, 0.1, 0.4  => 0.12762
# def combined_models(rx, rfx, xgbx):
#     return ((0.5 * best_ridge.predict(rx)) + 
#             (0.0 * best_rf.predict(rfx)) + 
#             (0.5 * best_xgb.predict(xgbx)) 
#            )
# y_pred_tr_en= combined_models(x_ridge_train  , x_rf_train  , x_xgb_train)
# y_pred_va_en= combined_models(x_ridge_valid  , x_rf_valid  , x_xgb_valid)
# y_pred_te_en= combined_models(x_ridge_predict, x_rf_predict, x_xgb_predict)

def combined_models(rx, rfx, xgbx, stackx):
    return ((0.40 * best_ridge.predict(rx)) + 
            (0.05 * best_rf.predict(rfx)) + 
            (0.05 * best_xgb.predict(xgbx)) +
            (0.50 * stack.predict(np.array(stackx)))
           )
y_pred_tr_en= combined_models(x_ridge_train  , x_rf_train  , x_xgb_train, x_stack_train)
y_pred_va_en= combined_models(x_ridge_valid  , x_rf_valid  , x_xgb_valid, x_stack_valid)
y_pred_te_en= combined_models(x_ridge_predict, x_rf_predict, x_xgb_predict, x_stack_predict)

In [ ]:
# y_train, y_pred_tr_ridge

In [ ]:
# 학습 데이터셋의 RMSE 계산 
rmse_tr_ridge = rmse(y_ridge_train, y_pred_tr_ridge)
rmse_tr_rf = rmse(y_rf_train, y_pred_tr_rf)
rmse_tr_xgb = rmse(y_xgb_train, y_pred_tr_xgb)
rmse_tr_en = rmse(y_xgb_train, y_pred_tr_en)
rmse_tr_en1 = rmse(y_rf_train, y_pred_tr_en1)

# 검증 데이터셋의 RMSE 계산 
rmse_va_ridge = rmse(y_ridge_valid, y_pred_va_ridge)
rmse_va_rf = rmse(y_rf_valid, y_pred_va_rf)
rmse_va_xgb = rmse(y_xgb_valid, y_pred_va_xgb)
rmse_va_en = rmse(y_xgb_valid, y_pred_va_en)
rmse_va_en1 = rmse(y_rf_valid, y_pred_va_en1)

# 검증 결과 Result DataFrame 저장
trynum = trynum + 1
index_name = list(str(trynum))
data = {
    'Work'                : [change_work],
    'RIDGE Train RSME'    : [round(rmse_tr_ridge, 4)], 
    'RIDGE valid RSME'    : [round(rmse_va_ridge, 4)],
    'RF Train RSME'       : [round(rmse_tr_rf, 4)], 
    'RF valid RSME'       : [round(rmse_va_rf, 4)],
    'XGB Train RSME'      : [round(rmse_tr_xgb, 4)], 
    'XGB valid RSME'      : [round(rmse_va_xgb, 4)],
    'RIDGE+XGB Train RSME': [round(rmse_tr_en, 4)], 
    'RIDGE+XGB valid RSME': [round(rmse_va_en, 4)],
    'RIDGE+RF Train RSME' : [round(rmse_tr_en1, 4)], 
    'RIDGE+RF valid RSME' : [round(rmse_va_en1, 4)]
}
df_tmp = pd.DataFrame(data, index = index_name)
df_results = pd.concat([df_results, df_tmp])
del df_tmp

In [ ]:
df_results

#### Filter 10.7 <= SalePrice <13.5,add flag field in train,add flag field in predict,numeric fillna in train,numeric fillna in predict,object fillna and convert to num in train,object fillna and convert to num in predict,add datefields in train,add datefields in predict,skewed data(skew > 0.35 or 0.4 ) np.log1p function before outlier process outlier in train,outlier in predict,stadardscaler in train,stadardscaler in predict,SEED=123,TEST_SIZE=0.15,Ridge : exceptfeature, outlierfeature remove, ridge np.logspace(-2,1,10)Rf : exceptfeature, outlierfeature remove, Rf : param : [1300],[8] XGB : exceptfeature, outlierfeature remove, XGB : param : [350, 360, 370],[0.1] 0.1164 0.1168 0.0499 0.1408 0.0073 0.1214 0.0599 0.1122 0.0789 0.1212, predict outlier 보정
=> kaggle : 0.12467

#### Filter 10.7 <= SalePrice <13.5,add flag field in train,add flag field in predict,numeric fillna in train,numeric fillna in predict,object fillna and convert to num in train,object fillna and convert to num in predict,add datefields in train,add datefields in predict,skewed data np.log1p function before outlier process outlier in train,outlier in predict,stadardscaler in train,stadardscaler in predict,SEED=123,TEST_SIZE=0.15,Ridge : exceptfeature, outlierfeature remove, ridge np.logspace(-2,1,10)Rf : exceptfeature, outlierfeature remove, Rf : param : [1300],[8] XGB : exceptfeature, outlierfeature remove, XGB : param : [350, 360, 370],[0.1]	0.1164	0.1168	0.0499	0.1408	0.0073	0.1214	0.0599	0.1122	0.0789	0.1212, predict outlier 보정
=> kaggle : 0.12467

#### Filter 10.7 <= SalePrice <13.5,add flag field in train,add flag field in predict,numeric fillna in train,numeric fillna in predict,object fillna and convert to num in train,object fillna and convert to num in predict,add datefields in train,add datefields in predict,skewed data np.log1p function before outlier process outlier in train,outlier in predict,stadardscaler in train,stadardscaler in predict,SEED=123,TEST_SIZE=0.15,Ridge : exceptfeature, outlierfeature remove, ridge np.logspace(-2,1,10)Rf : exceptfeature, outlierfeature remove, Rf : param : [1300],[8] XGB : exceptfeature, outlierfeature remove, XGB : param : [350, 360, 370],[0.1]	0.1164	0.1168	0.0499	0.1408	0.0073	0.1214	0.0599	0.1122	0.0789	0.1212
=> kaggle : 0.12709

#### Filter 10.7 <= SalePrice <13.5,add flag field in train,add flag field in predict,numeric fillna in train,numeric fillna in predict,object fillna and convert to num in train,object fillna and convert to num in predict,add datefields in train,add datefields in predict,outlier in train,outlier in predict,skewed data np.log1p function stadardscaler in train,stadardscaler in predict,SEED=123,TEST_SIZE=0.15,Ridge : exceptfeature, outlierfeature remove, ridge np.logspace(-2,1,10)Rf : exceptfeature, outlierfeature remove, Rf : param : [1300],[8] XGB : exceptfeature, outlierfeature remove, XGB : param : [350, 360, 370],[0.1]	0.1175	0.1188	0.0499	0.1409	0.0073	0.1213	0.0604	0.1139	0.0794	0.1227
=> kaggle : 0.12813

#### Filter 10.7 <= SalePrice <13.5,add flag field in train,add flag field in predict,numeric fillna in train,numeric fillna in predict,object fillna and convert to num in train,object fillna and convert to num in predict,add datefields in train,add datefields in predict,outlier in train,outlier in predict,stadardscaler in train,stadardscaler in predict,SEED=123,TEST_SIZE=0.15,Ridge : exceptfeature, outlierfeature remove, ridge np.logspace(-2,1,10)Rf : exceptfeature, outlierfeature remove, Rf : param : [1300] XGB : exceptfeature, outlierfeature remove, XGB : param : [350, 360, 370],[0.09, 0.1]	0.1164	0.1197	0.0499	0.1409	0.0073	0.1213	0.0599	0.1139	0.0787	0.1226
=> kaggle : 0.13124

#### Filter 10.7 <= SalePrice <13.5,add flag field in train,add flag field in predict,numeric fillna in train,numeric fillna in predict,object fillna and convert to num in train,object fillna and convert to num in predict,add datefields in train,add datefields in predict,outlier in train,outlier in predict,stadardscaler in train,stadardscaler in predict,SEED=123,TEST_SIZE=0.15,Ridge : exceptfeature, outlierfeature remove, ridge np.logspace(-2,1,10)Rf : exceptfeature, outlierfeature remove, Rf : param : [1200, 1300] XGB : exceptfeature, outlierfeature remove, XGB : param : [400, 410]	0.1164	0.1197	0.0499	0.1409	0.0051	0.1214	0.0593	0.1140	0.0787	0.1226
=> kaggle : 0.13124

#### Filter 10.7 <= SalePrice <13.5,add flag field in train,add flag field in predict,numeric fillna in train,numeric fillna in predict,object fillna and convert to num in train,object fillna and convert to num in predict,add datefields in train,add datefields in predict,outlier in train,outlier in predict,stadardscaler in train,stadardscaler in predict,SEED=123,TEST_SIZE=0.15,Ridge : exceptfeature, outlierfeature remove, ridge -2,1,10Rf : outlierfeature remove, XGB : outlierfeature remove, SEED=123,TEST_SIZE=0.15,Ridge : exceptfeature, outlierfeature remove, ridge -2,1,10Rf : exceptfeature, outlierfeature remove, Rf : param : [200, 250, 300, 500, 600] XGB : exceptfeature, outlierfeature remove, SEED=123,TEST_SIZE=0.15,Ridge : exceptfeature, outlierfeature remove, ridge -2,1,10Rf : exceptfeature, outlierfeature remove, Rf : param : [500, 800, 1000] XGB : exceptfeature, outlierfeature remove, XGB : param : [300, 400, 500]	0.1164	0.1197	0.0499	0.1413	0.0051	0.1214	0.0593	0.1140	0.0787	0.1228
=> kaggle : 0.13124

#### Filter 10.7 <= SalePrice <13.5,add flag field in train,add flag field in predict,numeric fillna in train,numeric fillna in predict,object fillna and convert to num in train,object fillna and convert to num in predict,add datefields in train,add datefields in predict,outlier in train,outlier in predict,stadardscaler in train,stadardscaler in predict,SEED=123,TEST_SIZE=0.15,Ridge : ALL, ridge -2,1,10Rf : All,XGB : All,	0.1164	0.1197	0.0501	0.1415	0.0195	0.1214	0.064	0.114	0.0786	0.1229
=> kaggle : 0.13132

### Filter 10.7<= SalePrice <13.5, TEST_SIZE=0.2, Ridge : ALL ridge -2,1,10 Rf : All XGB : All	0.1253	0.1380	0.0501	0.1449	0.0175	0.1290	0.0677	0.1235	0.0828	0.1322
=> kaggle : 0.14868

### Filter 10.7 <= SalePrice <13.5,SEED=123,TEST_SIZE=0.15,Ridge : ALL,del corr, ridge -2,1,10Rf : All,XGB : All,	0.1258	0.1303	0.0502	0.1431	0.0199	0.1253	0.0687	0.1195	0.0828	0.1292
=> kaggle : 0.1416

#### 5.6 검증용 데이터로 성능 비교

In [ ]:
rmse_valid = [rmse_va_ridge, rmse_va_rf, rmse_va_xgb, rmse_va_en]
recap_model = ['RIDGE', 'RF', 'XGB', 'Ensemble']
df_recap = pd.DataFrame({'Model': recap_model, 'Valid_RMSE' : rmse_valid})
ax = sns.barplot(x='Model', y='Valid_RMSE', data=df_recap, palette="Blues_d")
ax.set_ylim([0, 5]) 

In [ ]:
utils.plot_error('Ridge '+ str(trynum), y_ridge_valid, y_pred_va_ridge)
utils.plot_error('Random Forest ' + str(trynum), y_rf_valid, y_pred_va_rf)
utils.plot_error('XGBoost ' + str(trynum), y_xgb_valid, y_pred_va_xgb)
utils.plot_error('Ensenble ' + str(trynum), y_xgb_valid, y_pred_va_en)
utils.plot_error('Ensenble1 ' + str(trynum), y_rf_valid, y_pred_va_en1)

## 6. 결과제출(sumission)

### 6.1 예측값 저장하기 ( 지수함수 변환 포함 )

In [ ]:
# Log 함수 결과값을 원래의 값으로 환원하기 : np.expm1(b)
print(y_pred_te_en, np.expm1(y_pred_te_en))

In [ ]:
test['SalePrice'] = list(np.expm1(y_pred_te_en))
df_submission = test[['Id', 'SalePrice']] 
df_submission.head()

In [ ]:
# 예측값에 결측치가 포함되어 있는지 확인합니다.
df_submission.isnull().sum()

In [ ]:
# 예측값의 갯수가 평가용 데이터의 갯수와 동일한지 확인합니다.
assert len(df_submission) == len(test)
print(f'No. of Predict DataSet : {len(test)}\nNo. of Submission DataSet : {len(df_submission)}')

### 6.2 Predict Outlier 보정하기

In [ ]:
# Predict Outlier 보정하기 - 양극단 데이타 보정하기
q1 = df_submission['SalePrice'].quantile(0.0045)
q2 = df_submission['SalePrice'].quantile(0.99)
print(q1, q2)

In [ ]:
df_submission['SalePrice'] = df_submission['SalePrice'].apply(lambda x: x if x > q1 else x*0.77)
df_submission['SalePrice'] = df_submission['SalePrice'].apply(lambda x: x if x < q2 else x*1.1)

### 6.3 결과파일 저장하기

In [ ]:
# 예측 파일을 저장합니다. 
df_submission.set_index('Id', inplace=True)
df_submission.to_csv(f'submission.csv')